In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from numpy.random import multivariate_normal as mvnorm
from numpy.linalg import norm

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
%matplotlib inline

import pdb, time

seed = 0
np.random.seed(seed)

### Import and generate data

In [2]:
# Standardised Fama French 5 to industry portfolio 30
# Import data
data = pd.read_pickle("/Users/sikai/Downloads/ff5_30_standard_4000_draws.pkl")

# # Restrict to 100 time points
X = data[10:110].transpose(2, 1, 0)
X_cov = np.einsum('ijkl,jmkl->imkl', np.expand_dims(X, 1), np.expand_dims(X, 0))

### Train

In [ ]:
from regain.covariance import TaylorProxTimeGraphicalLasso, TimeGraphicalLasso
tic = time.perf_counter()
tgl = TaylorProxTimeGraphicalLasso(max_iter=3000, loss='LL', c_level=0.2, rho=1e2, theta=0.5, tol=1e-4, rtol=1e-4, psi="laplacian")
emp_inv_score_tp, baseline_score_tp, fit_score_tp, pre_tp = tgl.fit_cov(X_cov).eval_cov_pre() 
toc = time.perf_counter()
print('Running Time :{}'.format(toc - tic))
min_pre_tp = np.amin(np.abs(pre_tp[pre_tp != 0]))
print(min_pre_tp)

50940.731002451
50436.82788163196
50147.1613527367
49864.375888182825
49586.494015220305
49313.311848285215
> /Users/sikai/Dropbox/Research/post_sum/code/regain/regain/covariance/taylor_prox_time_graphical_lasso.py(210)taylor_prox_time_graphical_lasso()
-> out = minimize_scalar(_f, bounds=(0., 1.),  method='bounded')
(Pdb) loss_res_old
array([-9.3717846 , -9.638149  , -8.64117192, -9.26541924, -8.6514556 ,
       -9.03053568, -9.25489833, -8.64921378, -7.2610666 , -9.01845534,
       -8.94094379, -8.65606155, -8.74005592, -2.24329533, -8.94959412,
       -8.07963251, -8.54841906, -7.35693192, -8.01902318, -7.99483055,
       -8.18189401, -8.11137519, -7.49965806, -7.54180107, -1.93433225,
       -8.22070013, -8.16493439, -5.97981023, -7.69589528, -7.81657825,
       -7.44855534, -8.17211468,  0.59630586, -7.58596549, -7.53970131,
       -8.17685528, -7.14992736, -6.43755232, -5.14704921, -7.82877871,
       -7.67627213, -7.19584666, -7.6068428 , -6.29577857, -7.19522866,
       -7.3306

/Users/sikai/.pyenv/versions/3.7.6/lib/python3.7/site-packages/scipy/optimize/optimize.py:2116: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/Users/sikai/.pyenv/versions/3.7.6/lib/python3.7/site-packages/scipy/optimize/optimize.py:2117: RuntimeWarning: invalid value encountered in double_scalars
  p = (x - v) * tmp2 - (x - w) * tmp1


(Pdb) print(out.fun, out.x)
49.39289933680952 1.5036634767457785e-05


In [5]:
np.sum([sum(sum(abs(pre_tp[t]) == 0)) for t in range(X_cov.shape[-1])])

46

In [ ]:
np.sum([sum(sum(abs(pre_tp[t]) < 1e-4)) for t in range(X_cov.shape[-1])])

In [ ]:
from regain.covariance import InequalityTimeGraphicalLasso, TimeGraphicalLasso
res = {}
for i in [1.]:
    tic = time.perf_counter()
    tgl = InequalityTimeGraphicalLasso(max_iter=1000, loss='LL', c_level=0.2, c_prox='cvx', rho=1e2, div=i, tol=1e-8, rtol=1e-8, psi="laplacian")
    res[i] = tgl.fit_cov(X_cov).eval_cov_pre() 
    toc = time.perf_counter()
    print('Running Time :{}'.format(toc - tic))

In [ ]:
from regain.covariance import InequalityTimeGraphicalLasso, TimeGraphicalLasso
tic = time.perf_counter()
tgl = InequalityTimeGraphicalLasso(max_iter=5000, loss='LL', c_level=0.2, c_prox='grad', rho=2e2, theta=0.5, psi="laplacian")
emp_inv_score_grad, baseline_score_grad, fit_score_grad, pre_grad = tgl.fit_cov(X_cov).eval_cov_pre() 
toc = time.perf_counter()
print('Running Time :{}'.format(toc - tic))

In [ ]:
from regain.covariance import CVXInequalityTimeGraphicalLasso, TimeGraphicalLasso
pre_cvx = np.load("mosek_sol_ff5_30_standard_alpha_0.2.npy")

In [ ]:
from regain.norm import l1_od_norm
from regain.validation import check_norm_prox
psi, prox_psi, psi_node_penalty = check_norm_prox(tgl.psi)

def penalty_objective(Z_0, Z_1, Z_2, psi):
    """Penalty-only objective function for time-varying graphical LASSO."""
    return sum(map(l1_od_norm, Z_0)) + sum(map(psi, Z_2 - Z_1))

pre = {}
fit_score = {}
for i in [1e-2, 1e-3, 1e-4, 1e-6]:
# for i in [min_pre_tp]:
    pre[i] = np.array([k * (np.abs(k) >= i) for k in pre_cvx])
    tgl.precision_ = pre[i]
    emp_inv_score, baseline_score, fit_score[i], _ = tgl.eval_cov_pre() 
    print(penalty_objective(pre[i], pre[i][:-1], pre[i][:1], psi))

In [ ]:
tgl = TimeGraphicalLasso(alpha=1., beta=1., mode='admm', rho=1, tol=1e-4,
            rtol=1e-4, psi='laplacian', max_iter=3000, verbose=False, assume_centered=False, 
            return_history=False, update_rho_options=None, compute_objective=True, 
            stop_at=None, stop_when=1e-4, suppress_warn_list=False, init='empirical')
fit_score_, pre_ = tgl.fit_cov(X_cov).eval_cov_pre()   

In [ ]:
np.mean(fit_score_)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
colors = "rgm"
color_index = 0

idx = 50

ax.plot(range(X_cov.shape[0] * X_cov.shape[0]), tgl.emp_inv[idx].flatten(), color='k', label=r'Empirical Inverse')
for i in [min_pre_tp]:
# for i in [1e-2, 1e-3, 1e-4]:
    ax.plot(range(X_cov.shape[0] * X_cov.shape[0]), pre[i][idx].flatten(), color='r', alpha=0.5, 
            label=r'Constraint TGL SCS, Thres = {}'.format(i))
# for i in [2.]:    
#     ax.plot(range(X_cov.shape[0] * X_cov.shape[0]), res[i][3][idx].flatten(), color='g', alpha=0.5,
#             label=r'Constraint TGL ADMM CVX, Div = {}'.format(i))
ax.plot(range(X_cov.shape[0] * X_cov.shape[0]), pre_grad[idx].flatten(), color='m', alpha=0.5,
        label=r'Constraint TGL ADMM Gradient')
ax.plot(range(X_cov.shape[0] * X_cov.shape[0]), pre_tp[idx].flatten(), color='g', alpha=0.5,
        label=r'Constraint TGL ADMM Linear')
ax.plot(range(X_cov.shape[0] * X_cov.shape[0]), pre_[idx].flatten(), color='y', label=r'Vanilla TGL')


fig.legend(fontsize=15)
ax.set_ylabel('Values', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Entries', fontsize=15)
ax.set_title(r'Precisions at t={} for Empirical Inverse, TGL and Constraint TGL SCS/ADMM'.format(idx), fontsize=20)
plt.tight_layout()
# plt.savefig('pre_{}_{}_{}_admm_relax_taylor.pdf'.format(idx, dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
colors = "rgm"
color_index = 0

for i in [min_pre_tp]:
# for i in [1e-2, 1e-3, 1e-4]:
    diff = (pre[i] -  pre_cvx).flatten()
    ax.hist(diff, bins=100, color='r', alpha=0.5, 
            label=r'Constraint TGL SCS, Thres = {}'.format(i))
# for i in [2.]:    
#     diff = (res[i][3] -  pre_cvx).flatten()
#     ax.hist(diff, bins=50, color='g', alpha=0.2,
#             label=r'Constraint TGL ADMM CVX, Div = {}'.format(i))
diff = (pre_grad -  pre_cvx).flatten()
ax.hist(diff, bins=100, color='m', alpha=0.2,
        label=r'Constraint TGL ADMM Gradient')
diff = (pre_tp -  pre_cvx).flatten()
ax.hist(diff, bins=100, color='g', alpha=0.2,
        label=r'Constraint TGL ADMM Linear')
# diff = (pre_ -  pre_cvx).flatten()
# ax.hist(diff, bins=100, color='y', alpha=0.5, 
#         label=r'Vanilla TGL')

fig.legend(fontsize=15)
ax.set_ylabel('Counts', fontsize=15)
ax.set_xlabel('Values', fontsize=15)
major_loc = MultipleLocator(base=0.1)
ax.xaxis.set_major_locator(major_loc)
ax.set_xlim((-1, 1))
# ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_title(r'Difference in Precisions for TGL and Constraint TGL SCS/ADMM', fontsize=20)
plt.tight_layout()
# plt.savefig('pre_diff_hist_{}_{}_taylor.pdf'.format(dim_X, dim_fac))

In [ ]:
res[1][0]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
colors = "rgb"
color_index = 0
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)

ax.plot(range(X_cov.shape[-1]), res[1][0], color='k', label=r'Empirical Inverse')
ax.plot(range(X_cov.shape[-1]), res[1][1], color='c', label=r'Constraint')
# for i in [min_pre_tp]:
for i in [1e-2, 1e-3, 1e-4]:
    mean_diff = np.mean(np.array(fit_score[i]) - baseline_score)
#     ax.plot(range(X_cov.shape[-1]), fit_score[i], color='r', alpha=0.5, 
    ax.plot(range(X_cov.shape[-1]), fit_score[i], color=colors[color_index], alpha=0.5, 
            label=r'Constraint TGL SCS, Thres = {}, Mean Diff = {:.3f}'.format(i, mean_diff))
    color_index += 1
# # for i in [2.]:    
# #     mean_diff = np.mean(np.array(res[i][2]) - res[i][1])
# #     ax.plot(range(X_cov.shape[-1]), res[i][2], alpha=0.5, color='g',
# #             label=r'Constraint TGL ADMM CVX, Div = {}, Mean Diff = {:.3f}'.format(i, mean_diff))
# # mean_diff = np.mean(np.array(fit_score_diff) - baseline_score)
# # ax.plot(range(n_time), fit_score_diff, alpha=0.5,
# #         label=r'Constraint TGL ADMM Difference, Mean Diff = {:.3f}'.format(mean_diff))
mean_diff = np.mean(np.array(fit_score_grad) - baseline_score_grad)
ax.plot(range(X_cov.shape[-1]), fit_score_grad, alpha=0.5, color='m',
        label=r'Constraint TGL ADMM Gradient, Mean Diff = {:.3f}'.format(mean_diff))
# mean_diff = np.mean(np.array(fit_score_tp) - baseline_score_tp)
# print(np.sum((np.array(fit_score_tp) - baseline_score_tp) > 0.01))
# ax.plot(range(X_cov.shape[-1]), fit_score_tp, alpha=0.5, color='g',
#         label=r'Constraint TGL ADMM Linear, Mean Diff = {:.3f}'.format(mean_diff))
# mean_diff = np.mean(np.array(fit_score_) - res[1][1])
# print(np.sum((np.array(fit_score_) - res[1][1]) > 0.01))
# ax.plot(range(X_cov.shape[-1]), fit_score_, color='y', label=r'Vanilla TGL, Mean Diff = {:.3f}'.format(mean_diff))

fig.legend(fontsize=15)
ax.set_ylabel('Negative Log Likelihood', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Negative Log Likelihood for Empirical Inverse, Constraint, TGL and Constraint TGL SCS/ADMM', fontsize=20)
# ax.set_title(r'Negative Log Likelihood for Empirical Inverse, Constraint, TGL and Constraint TGL SCS', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_like_{}_{}_admm_taylor.pdf'.format(dim_X, dim_fac))
# plt.savefig('diff_like_{}_{}_scs.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
colors = "rgb"
color_index = 0

ax.plot(range(X_cov.shape[-1]), [sum(sum(abs(tgl.emp_inv[k]) > 0)) for k in range(X_cov.shape[-1])], 
        color='k', label=r'Empirical Inverse')
# for i in [min_pre_tp]:
for i in [1e-2, 1e-3, 1e-4]:
    supp = [sum(sum(abs(pre[i][t]) > 0)) for t in range(X_cov.shape[-1])]
    mean_supp = np.mean(supp)
#     ax.plot(range(X_cov.shape[-1]), supp, color='r', alpha=0.5, 
    ax.plot(range(X_cov.shape[-1]), supp, color=colors[color_index], alpha=0.5, 
            label=r'Constraint TGL SCS, Thres = {}, Mean Supp = {}'.format(i, mean_supp))
    color_index += 1
# # for i in [2.]:    
# #     supp = [sum(sum(abs(res[i][3][t]) > 0)) for t in range(X_cov.shape[-1])]
# #     mean_supp = np.mean(supp)
# #     ax.plot(range(X_cov.shape[-1]), supp, color='g', alpha=0.5,
# #             label=r'Constraint TGL ADMM CVX, Div = {}, Mean Supp = {:.3f}'.format(i, mean_supp))
supp = [sum(sum(abs(pre_grad[t]) > 0)) for t in range(X_cov.shape[-1])]
ax.plot(range(X_cov.shape[-1]), supp, color='m', alpha=0.5,
        label=r'Constraint TGL ADMM Gradient, Mean Supp = {:.3f}'.format(np.mean(supp)))
# supp = [sum(sum(abs(pre_tp[t]) > 0)) for t in range(X_cov.shape[-1])]
# ax.plot(range(X_cov.shape[-1]), supp, color='g', alpha=0.5,
#         label=r'Constraint TGL ADMM Linear, Mean Supp = {:.3f}'.format(np.mean(supp)))
supp = [sum(sum(abs(pre_[t]) > 0)) for t in range(X_cov.shape[-1])]
ax.plot(range(X_cov.shape[-1]), [sum(sum(abs(pre_[k]) > 0)) for k in range(X_cov.shape[-1])], 
        color='y', label=r'Vanilla TGL, , Mean Supp = {:.3f}'.format(np.mean(supp)))


fig.legend(fontsize=15)
ax.set_ylabel('Support', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Support for Empirical Inverse, TGL and Constraint TGL SCS/ADMM', fontsize=20)
# ax.set_title(r'Support for Empirical Inverse, TGL and Constraint TGL SCS', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_supp_{}_{}_admm_taylor.pdf'.format(dim_X, dim_fac))
# plt.savefig('diff_supp_{}_{}_scs.pdf'.format(dim_X, dim_fac))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fig.patch.set_facecolor('white')
colors = "rgb"
color_index = 0

diff_emp = [norm(tgl.emp_inv[t] - tgl.emp_inv[t-1], 'fro') for t in range(1, X_cov.shape[-1])]
ax.plot(range(1, X_cov.shape[-1]), np.array(diff_emp), alpha=1, color='k', label=r'Empirical Inverse')
# for i in [min_pre_tp]:
for i in [1e-2, 1e-3, 1e-4]:
    diff = [norm(pre[i][t] - pre[i][t-1], 'fro') for t in range(1, X_cov.shape[-1])]
    mean_diff = np.mean(diff)
#     ax.plot(range(1, X_cov.shape[-1]), diff, color='r', alpha=0.5, 
    ax.plot(range(1, X_cov.shape[-1]), diff, color=colors[color_index], alpha=0.5, 
            label=r'Constraint TGL SCS, Thres = {}, Mean Diff = {:.3f}'.format(i, mean_diff))
    color_index += 1
# # for i in [2.]:    
# #     diff_cvx = [norm(res[i][3][t] - res[i][3][t-1], 'fro') for t in range(1, X_cov.shape[-1])]
# #     ax.plot(range(1, X_cov.shape[-1]), diff_cvx, color='g', alpha=0.5,
# #             label=r'Constraint TGL ADMM CVX, Div = {}, Mean Diff = {:.3f}'.format(i, np.mean(diff_cvx)))
diff_grad = [norm(pre_grad[t] - pre_grad[t-1], 'fro') for t in range(1, X_cov.shape[-1])]
ax.plot(range(1, X_cov.shape[-1]), diff_grad, color='m', alpha=0.5,
        label=r'Constraint TGL ADMM Gradient, Mean Diff = {:.3f}'.format(np.mean(diff_grad)))
# diff_grad = [norm(pre_tp[t] - pre_tp[t-1], 'fro') for t in range(1, X_cov.shape[-1])]
# ax.plot(range(1, X_cov.shape[-1]), diff_grad, color='g', alpha=0.5,
#         label=r'Constraint TGL ADMM Linear, Mean Diff = {:.3f}'.format(np.mean(diff_grad)))
diff_fit_ = [norm(pre_[t] - pre_[t-1], 'fro') for t in range(1, X_cov.shape[-1])]
ax.plot(range(1, X_cov.shape[-1]), np.array(diff_fit_), color='y', alpha=1, 
        label=r'Vanilla TGL, Mean Diff = {:.3f}'.format(np.mean(diff_fit_)))

        
fig.legend(fontsize=15)
ax.set_ylabel('Difference in Frobenius Norm', fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlabel('Time t', fontsize=15)
ax.set_title(r'Difference in Frobenius Norm for Empirical Inverse, TGL and Constraint TGL SCS/ADMM', fontsize=20)
# ax.set_title(r'Difference in Frobenius Norm for Empirical Inverse, TGL and Constraint TGL SCS', fontsize=20)
plt.tight_layout()
# plt.savefig('diff_fro_{}_{}_admm_taylor.pdf'.format(dim_X, dim_fac))
# plt.savefig('diff_fro_{}_{}_scs.pdf'.format(dim_X, dim_fac))